In [36]:
import pandas as pd
import numpy as np
import os
import datetime
from scipy.spatial.distance import cdist
import geopandas as gpd
from shapely.geometry import Point
from sklearn.neighbors import BallTree
from geopy import distance

In [37]:
import git
import sys
repo = git.Repo("./", search_parent_directories=True)
homedir = repo.working_dir

In [38]:
#Fixed Months, no need to run this f"{homedir}/data/us/air_quality/1.tgz"
Air_Qual_1 = pd.read_csv(f"{homedir}/data/us/air_quality/1.tgz", compression='gzip')                 
Air_Qual_2 = pd.read_csv(f"{homedir}/data/us/air_quality/2.tgz", compression='gzip')                 
Air_Qual_3 = pd.read_csv(f"{homedir}/data/us/air_quality/3.tgz", compression='gzip')                 

In [39]:
#Air Data that Changes Weekly
Air_Qual_4 = pd.read_csv(f"{homedir}/data/us/air_quality/4.tgz", compression='gzip')                 

In [40]:
#Loading in County_Centers Data to map Air Quality to FIPs code
County_Centers = pd.read_csv(f"{homedir}/data/us/geolocation/county_centers.csv")
County_Centers = County_Centers.drop(columns=['clon00','clat00','pclon00','pclat00','clon10','clat10'])
#Removing NaN rows
County_Centers = County_Centers.dropna()

In [41]:
def closest_point(point, points):
    """ Find closest point from a list of points. """
    return points[cdist([point], points).argmin()]

In [42]:
def match_value(df, col1, x, col2):
    """ Match value x from col1 row to value in col2. """
    return df[df[col1] == x][col2].values[0]

In [43]:
def air_qual_county(fips, date, gas_data, colname, County_Centers, neighbors=5):
    if date == '01/01/20':
        print(fips)
    #Gets the coordinates of this fips code county
    Center = np.array(County_Centers[County_Centers['fips'] == fips][['pclat10','pclon10']].reset_index(drop=True).loc[0]).reshape(1,-1)
    #get only relevant date data
    gas_data = gas_data[gas_data['ValidDate'] == date]
    #Making geo dataframe to do distance calculations
    Geo = gpd.GeoDataFrame({
        'geometry': (a, b),
        'x': float(a),
        'y': float(b),
        'val': c,
    } for a, b, c in zip(gas_data['Latitude'], gas_data['Longitude'], gas_data[colname]))
    #distance tree
    if len(Geo) == 0:
        return 0
    tree = BallTree(Geo[['x','y']].values, leaf_size=2) 
    #Querying for the nearest k neighbors, we use 5 as default
    dist, ids = tree.query(Center, k = neighbors) # The number of nearest neighbors
    #list of the nearest neighbors to our county
    near = Geo.loc[ids.tolist()[0]]
    totalinvdist = 0
    weightedval = 0
    extrainvdist = 0
    extraweightedval = 0
    for ind in near.index:
        nearloc = near.loc[ind]['geometry']
        if abs(nearloc[0]) <= 90 and abs(nearloc[1]) <= 180:
            dist = distance.distance(Center, near.loc[ind]['geometry']).km
            #If possible we only want nearby influences, so we cap this at 100km
            if dist < 100:
                totalinvdist += (1/dist)**1
                weightedval += ((1/dist)**1)*near.loc[ind]['val']
            #If no closeby measurments are available, then further away data is used
            else: 
                extrainvdist += (1/dist)**1
                extraweightedval += ((1/dist)**1)*near.loc[ind]['val']
    if totalinvdist == 0:
        if extrainvdist == 0:
            return 0
        else:
            return extraweightedval/extrainvdist
    else:
        return weightedval/totalinvdist

In [44]:
#Getting only US Air Data
Air_Qual_1 = Air_Qual_1[Air_Qual_1['CountryCode'] == 'US']
Air_Qual_2 = Air_Qual_2[Air_Qual_2['CountryCode'] == 'US']
Air_Qual_3 = Air_Qual_3[Air_Qual_3['CountryCode'] == 'US']
Air_Qual_4 = Air_Qual_4[Air_Qual_4['CountryCode'] == 'US']
#Dropping unneeded columns
Air_Qual_1 = Air_Qual_1.drop(columns=['out/','SiteName','GMTOffset','CountryCode','StateName', 'Elevation', \
                                      'DataSource','ReportingArea_PipeDelimited','Status','EPARegion'])
Air_Qual_2 = Air_Qual_2.drop(columns=['out/','SiteName','GMTOffset','CountryCode','StateName', 'Elevation', \
                                      'DataSource','ReportingArea_PipeDelimited','Status','EPARegion'])
Air_Qual_3 = Air_Qual_3.drop(columns=['out/','SiteName','GMTOffset','CountryCode','StateName', 'Elevation', \
                                      'DataSource','ReportingArea_PipeDelimited','Status','EPARegion'])
Air_Qual_4 = Air_Qual_4.drop(columns=['out/','SiteName','GMTOffset','CountryCode','StateName', 'Elevation', \
                                      'DataSource','ReportingArea_PipeDelimited','Status','EPARegion'])

Air_Qual_1 = Air_Qual_1.astype({'Latitude': 'float64','Longitude': 'float64'})
Air_Qual_2 = Air_Qual_2.astype({'Latitude': 'float64','Longitude': 'float64'})
Air_Qual_3 = Air_Qual_3.astype({'Latitude': 'float64','Longitude': 'float64'})
Air_Qual_4 = Air_Qual_4.astype({'Latitude': 'float64','Longitude': 'float64'})

#joining data together
Air_Qual = Air_Qual_1.append(Air_Qual_2, ignore_index = True) 
Air_Qual = Air_Qual.append(Air_Qual_3, ignore_index = True) 
Air_Qual = Air_Qual.append(Air_Qual_4, ignore_index = True) 



In [45]:
#Cleaning Air_Qual to split it up
Air_Qual = Air_Qual.drop(columns=['ValidTime','OZONE_Measured','PM10_Measured','PM25_Measured','NO2_Measured','PM25',\
                                  'PM25_Unit', 'OZONE','OZONE_Unit','NO2','NO2_Unit','PM10','PM10_Unit'])
Air_Qual.to_csv('Air_Qual.csv')

In [47]:
#This is a list of all the counties and dates
County_List = list(County_Centers.fips.unique())
Date_List = list(Air_Qual.ValidDate.unique())
#This creates a base dataframe that contains all pairs of FIPS codes with the valid dates given in Air_Qual
CL, DL = pd.core.reshape.util.cartesian_product([County_List, Date_List])
BaseFrame = pd.DataFrame(dict(FIPS=CL, Date=DL)).sort_values(['FIPS','Date']).reset_index(drop=True)

In [48]:
#Making individual Dataframes for each type of Gas particle collected
#Doing Ozone First
Ozone = Air_Qual.loc[Air_Qual['OZONE_AQI'].astype(float) >= -1]
Ozone = Ozone.drop(columns=['PM10_AQI','PM25_AQI','NO2_AQI','CO','CO_Unit','SO2','SO2_Unit'])
#Setting the datapoint to be a float
Ozone = Ozone.astype({'OZONE_AQI': 'float64'}).sort_values(['Latitude','Longitude','ValidDate'])
#Grouping data together, for each set of idential Latitude and longitude coords with same date, the values are averaged
Ozone = Ozone.groupby(['Latitude','Longitude','ValidDate']).mean().reset_index()
#Making County_Ozone data from BaseFrame
County_Ozone = BaseFrame.copy()
#creating new 
County_Ozone['OZONE_AQI'] = County_Ozone.apply(lambda x: air_qual_county(x['FIPS'], x['Date'], Ozone, 'OZONE_AQI', County_Centers), axis = 1)

1001
1003
1005
1007
1009
1011
1013
1015
1017
1019
1021
1023
1025
1027
1029
1031
1033
1035
1037
1039
1041
1043
1045
1047
1049
1051
1053
1055
1057
1059
1061
1063
1065
1067
1069
1071
1073
1075
1077
1079
1081
1083
1085
1087
1089
1091
1093
1095
1097
1099
1101
1103
1105
1107
1109
1111
1113
1115
1117
1119
1121
1123
1125
1127
1129
1131
1133
2013
2016
2020
2050
2060
2068
2070
2090
2100
2105
2110
2122
2130
2150
2164
2170
2180
2185
2188
2195
2198
2220
2230
2240
2261
2270
2275
2282
2290
4001
4003
4005
4007
4009
4011
4012
4013
4015
4017
4019
4021
4023
4025
4027
5001
5003
5005
5007
5009
5011
5013
5015
5017
5019
5021
5023
5025
5027
5029
5031
5033
5035
5037
5039
5041
5043
5045
5047
5049
5051
5053
5055
5057
5059
5061
5063
5065
5067
5069
5071
5073
5075
5077
5079
5081
5083
5085
5087
5089
5091
5093
5095
5097
5099
5101
5103
5105
5107
5109
5111
5113
5115
5117
5119
5121
5123
5125
5127
5129
5131
5133
5135
5137
5139
5141
5143
5145
5147
5149
6001
6003
6005
6007
6009
6011
6013
6015
6017
6019
6021
6023
6025
6027


28037
28039
28041
28043
28045
28047
28049
28051
28053
28055
28057
28059
28061
28063
28065
28067
28069
28071
28073
28075
28077
28079
28081
28083
28085
28087
28089
28091
28093
28095
28097
28099
28101
28103
28105
28107
28109
28111
28113
28115
28117
28119
28121
28123
28125
28127
28129
28131
28133
28135
28137
28139
28141
28143
28145
28147
28149
28151
28153
28155
28157
28159
28161
28163
29001
29003
29005
29007
29009
29011
29013
29015
29017
29019
29021
29023
29025
29027
29029
29031
29033
29035
29037
29039
29041
29043
29045
29047
29049
29051
29053
29055
29057
29059
29061
29063
29065
29067
29069
29071
29073
29075
29077
29079
29081
29083
29085
29087
29089
29091
29093
29095
29097
29099
29101
29103
29105
29107
29109
29111
29113
29115
29117
29119
29121
29123
29125
29127
29129
29131
29133
29135
29137
29139
29141
29143
29145
29147
29149
29151
29153
29155
29157
29159
29161
29163
29165
29167
29169
29171
29173
29175
29177
29179
29181
29183
29185
29186
29187
29189
29195
29197
29199
29201
29203
29205
2920

49017
49019
49021
49023
49025
49027
49029
49031
49033
49035
49037
49039
49041
49043
49045
49047
49049
49051
49053
49055
49057
50001
50003
50005
50007
50009
50011
50013
50015
50017
50019
50021
50023
50025
50027
51001
51003
51005
51007
51009
51011
51013
51015
51017
51019
51021
51023
51025
51027
51029
51031
51033
51035
51036
51037
51041
51043
51045
51047
51049
51051
51053
51057
51059
51061
51063
51065
51067
51069
51071
51073
51075
51077
51079
51081
51083
51085
51087
51089
51091
51093
51095
51097
51099
51101
51103
51105
51107
51109
51111
51113
51115
51117
51119
51121
51125
51127
51131
51133
51135
51137
51139
51141
51143
51145
51147
51149
51153
51155
51157
51159
51161
51163
51165
51167
51169
51171
51173
51175
51177
51179
51181
51183
51185
51187
51191
51193
51195
51197
51199
51510
51515
51520
51530
51540
51550
51570
51580
51590
51595
51600
51610
51620
51630
51640
51650
51660
51670
51678
51680
51683
51685
51690
51700
51710
51720
51730
51735
51740
51750
51760
51770
51775
51790
51800
51810
5182

In [ ]:
County_Ozone

In [52]:
County_Ozone.to_csv('County_Ozone_Filled.csv')

In [50]:
#Now doing PM10
PM10 = Air_Qual.loc[Air_Qual['PM10_AQI'].astype(float) >= -1]
PM10 = PM10.drop(columns=['OZONE_AQI','PM25_AQI','NO2_AQI','CO','CO_Unit','SO2','SO2_Unit'])
PM10 = PM10.astype({'PM10_AQI': 'float64'}).sort_values(['Latitude','Longitude','ValidDate'])
#Grouping data together, for each set of idential Latitude and longitude coords with same date, the values are averaged
PM10 = PM10.groupby(['Latitude','Longitude','ValidDate']).mean().reset_index()
#Making County_Ozone data from BaseFrame
County_PM10 = BaseFrame.copy()
#creating new 
County_PM10['PM10_AQI'] = County_PM10.apply(lambda x: air_qual_county(x['FIPS'], x['Date'], PM10, 'PM10_AQI', County_Centers, neighbors=10), axis = 1)


1001
1003
1005
1007
1009
1011
1013
1015
1017
1019
1021
1023
1025
1027
1029
1031
1033
1035
1037
1039
1041
1043
1045
1047
1049
1051
1053
1055
1057
1059
1061
1063
1065
1067
1069
1071
1073
1075
1077
1079
1081
1083
1085
1087
1089
1091
1093
1095
1097
1099
1101
1103
1105
1107
1109
1111
1113
1115
1117
1119
1121
1123
1125
1127
1129
1131
1133
2013
2016
2020
2050
2060
2068
2070
2090
2100
2105
2110
2122
2130
2150
2164
2170
2180
2185
2188
2195
2198
2220
2230
2240
2261
2270
2275
2282
2290
4001
4003
4005
4007
4009
4011
4012
4013
4015
4017
4019
4021
4023
4025
4027
5001
5003
5005
5007
5009
5011
5013
5015
5017
5019
5021
5023
5025
5027
5029
5031
5033
5035
5037
5039
5041
5043
5045
5047
5049
5051
5053
5055
5057
5059
5061
5063
5065
5067
5069
5071
5073
5075
5077
5079
5081
5083
5085
5087
5089
5091
5093
5095
5097
5099
5101
5103
5105
5107
5109
5111
5113
5115
5117
5119
5121
5123
5125
5127
5129
5131
5133
5135
5137
5139
5141
5143
5145
5147
5149
6001
6003
6005
6007
6009
6011
6013
6015
6017
6019
6021
6023
6025
6027


28037
28039
28041
28043
28045
28047
28049
28051
28053
28055
28057
28059
28061
28063
28065
28067
28069
28071
28073
28075
28077
28079
28081
28083
28085
28087
28089
28091
28093
28095
28097
28099
28101
28103
28105
28107
28109
28111
28113
28115
28117
28119
28121
28123
28125
28127
28129
28131
28133
28135
28137
28139
28141
28143
28145
28147
28149
28151
28153
28155
28157
28159
28161
28163
29001
29003
29005
29007
29009
29011
29013
29015
29017
29019
29021
29023
29025
29027
29029
29031
29033
29035
29037
29039
29041
29043
29045
29047
29049
29051
29053
29055
29057
29059
29061
29063
29065
29067
29069
29071
29073
29075
29077
29079
29081
29083
29085
29087
29089
29091
29093
29095
29097
29099
29101
29103
29105
29107
29109
29111
29113
29115
29117
29119
29121
29123
29125
29127
29129
29131
29133
29135
29137
29139
29141
29143
29145
29147
29149
29151
29153
29155
29157
29159
29161
29163
29165
29167
29169
29171
29173
29175
29177
29179
29181
29183
29185
29186
29187
29189
29195
29197
29199
29201
29203
29205
2920

49017
49019
49021
49023
49025
49027
49029
49031
49033
49035
49037
49039
49041
49043
49045
49047
49049
49051
49053
49055
49057
50001
50003
50005
50007
50009
50011
50013
50015
50017
50019
50021
50023
50025
50027
51001
51003
51005
51007
51009
51011
51013
51015
51017
51019
51021
51023
51025
51027
51029
51031
51033
51035
51036
51037
51041
51043
51045
51047
51049
51051
51053
51057
51059
51061
51063
51065
51067
51069
51071
51073
51075
51077
51079
51081
51083
51085
51087
51089
51091
51093
51095
51097
51099
51101
51103
51105
51107
51109
51111
51113
51115
51117
51119
51121
51125
51127
51131
51133
51135
51137
51139
51141
51143
51145
51147
51149
51153
51155
51157
51159
51161
51163
51165
51167
51169
51171
51173
51175
51177
51179
51181
51183
51185
51187
51191
51193
51195
51197
51199
51510
51515
51520
51530
51540
51550
51570
51580
51590
51595
51600
51610
51620
51630
51640
51650
51660
51670
51678
51680
51683
51685
51690
51700
51710
51720
51730
51735
51740
51750
51760
51770
51775
51790
51800
51810
5182

In [54]:
County_PM10.to_csv('County_PM10_Filled.csv')

In [55]:
#Now doing PM25
PM25 = Air_Qual.loc[Air_Qual['PM25_AQI'].astype(float) >= -1]
PM25 = PM25.drop(columns=['OZONE_AQI','PM10_AQI','NO2_AQI','CO','CO_Unit','SO2','SO2_Unit'])
PM25 = PM25.astype({'PM25_AQI': 'float64'}).sort_values(['Latitude','Longitude','ValidDate'])
PM25 = PM25.groupby(['Latitude','Longitude','ValidDate']).mean().reset_index()
County_PM25 = BaseFrame.copy()
County_PM25['PM25_AQI'] = County_PM25.apply(lambda x: air_qual_county(x['FIPS'], x['Date'], PM25, 'PM25_AQI', County_Centers, neighbors=10), axis = 1)


1001
1003
1005
1007
1009
1011
1013
1015
1017
1019
1021
1023
1025
1027
1029
1031
1033
1035
1037
1039
1041
1043
1045
1047
1049
1051
1053
1055
1057
1059
1061
1063
1065
1067
1069
1071
1073
1075
1077
1079
1081
1083
1085
1087
1089
1091
1093
1095
1097
1099
1101
1103
1105
1107
1109
1111
1113
1115
1117
1119
1121
1123
1125
1127
1129
1131
1133
2013
2016
2020
2050
2060
2068
2070
2090
2100
2105
2110
2122
2130
2150
2164
2170
2180
2185
2188
2195
2198
2220
2230
2240
2261
2270
2275
2282
2290
4001
4003
4005
4007
4009
4011
4012
4013
4015
4017
4019
4021
4023
4025
4027
5001
5003
5005
5007
5009
5011
5013
5015
5017
5019
5021
5023
5025
5027
5029
5031
5033
5035
5037
5039
5041
5043
5045
5047
5049
5051
5053
5055
5057
5059
5061
5063
5065
5067
5069
5071
5073
5075
5077
5079
5081
5083
5085
5087
5089
5091
5093
5095
5097
5099
5101
5103
5105
5107
5109
5111
5113
5115
5117
5119
5121
5123
5125
5127
5129
5131
5133
5135
5137
5139
5141
5143
5145
5147
5149
6001
6003
6005
6007
6009
6011
6013
6015
6017
6019
6021
6023
6025
6027


28037
28039
28041
28043
28045
28047
28049
28051
28053
28055
28057
28059
28061
28063
28065
28067
28069
28071
28073
28075
28077
28079
28081
28083
28085
28087
28089
28091
28093
28095
28097
28099
28101
28103
28105
28107
28109
28111
28113
28115
28117
28119
28121
28123
28125
28127
28129
28131
28133
28135
28137
28139
28141
28143
28145
28147
28149
28151
28153
28155
28157
28159
28161
28163
29001
29003
29005
29007
29009
29011
29013
29015
29017
29019
29021
29023
29025
29027
29029
29031
29033
29035
29037
29039
29041
29043
29045
29047
29049
29051
29053
29055
29057
29059
29061
29063
29065
29067
29069
29071
29073
29075
29077
29079
29081
29083
29085
29087
29089
29091
29093
29095
29097
29099
29101
29103
29105
29107
29109
29111
29113
29115
29117
29119
29121
29123
29125
29127
29129
29131
29133
29135
29137
29139
29141
29143
29145
29147
29149
29151
29153
29155
29157
29159
29161
29163
29165
29167
29169
29171
29173
29175
29177
29179
29181
29183
29185
29186
29187
29189
29195
29197
29199
29201
29203
29205
2920

49017
49019
49021
49023
49025
49027
49029
49031
49033
49035
49037
49039
49041
49043
49045
49047
49049
49051
49053
49055
49057
50001
50003
50005
50007
50009
50011
50013
50015
50017
50019
50021
50023
50025
50027
51001
51003
51005
51007
51009
51011
51013
51015
51017
51019
51021
51023
51025
51027
51029
51031
51033
51035
51036
51037
51041
51043
51045
51047
51049
51051
51053
51057
51059
51061
51063
51065
51067
51069
51071
51073
51075
51077
51079
51081
51083
51085
51087
51089
51091
51093
51095
51097
51099
51101
51103
51105
51107
51109
51111
51113
51115
51117
51119
51121
51125
51127
51131
51133
51135
51137
51139
51141
51143
51145
51147
51149
51153
51155
51157
51159
51161
51163
51165
51167
51169
51171
51173
51175
51177
51179
51181
51183
51185
51187
51191
51193
51195
51197
51199
51510
51515
51520
51530
51540
51550
51570
51580
51590
51595
51600
51610
51620
51630
51640
51650
51660
51670
51678
51680
51683
51685
51690
51700
51710
51720
51730
51735
51740
51750
51760
51770
51775
51790
51800
51810
5182

In [56]:
County_PM25.to_csv('County_PM25_Filled.csv')

In [57]:
#Now doing NO2
NO2 = Air_Qual.loc[Air_Qual['NO2_AQI'].astype(float) >= -1]
NO2 = NO2.drop(columns=['OZONE_AQI','PM10_AQI','PM25_AQI','CO','CO_Unit','SO2','SO2_Unit'])
NO2 = NO2.astype({'NO2_AQI': 'float64'}).sort_values(['Latitude','Longitude','ValidDate'])
NO2 = NO2.groupby(['Latitude','Longitude','ValidDate']).mean().reset_index()
County_NO2 = BaseFrame.copy()
County_NO2['NO2_AQI'] = County_NO2.apply(lambda x: air_qual_county(x['FIPS'], x['Date'], NO2, 'NO2_AQI', County_Centers, neighbors=10), axis = 1)
County_NO2.to_csv('County_NO2.csv')

1001
1003
1005
1007
1009
1011
1013
1015
1017
1019
1021
1023
1025
1027
1029
1031
1033
1035
1037
1039
1041
1043
1045
1047
1049
1051
1053
1055
1057
1059
1061
1063
1065
1067
1069
1071
1073
1075
1077
1079
1081
1083
1085
1087
1089
1091
1093
1095
1097
1099
1101
1103
1105
1107
1109
1111
1113
1115
1117
1119
1121
1123
1125
1127
1129
1131
1133
2013
2016
2020
2050
2060
2068
2070
2090
2100
2105
2110
2122
2130
2150
2164
2170
2180
2185
2188
2195
2198
2220
2230
2240
2261
2270
2275
2282
2290
4001
4003
4005
4007
4009
4011
4012
4013
4015
4017
4019
4021
4023
4025
4027
5001
5003
5005
5007
5009
5011
5013
5015
5017
5019
5021
5023
5025
5027
5029
5031
5033
5035
5037
5039
5041
5043
5045
5047
5049
5051
5053
5055
5057
5059
5061
5063
5065
5067
5069
5071
5073
5075
5077
5079
5081
5083
5085
5087
5089
5091
5093
5095
5097
5099
5101
5103
5105
5107
5109
5111
5113
5115
5117
5119
5121
5123
5125
5127
5129
5131
5133
5135
5137
5139
5141
5143
5145
5147
5149
6001
6003
6005
6007
6009
6011
6013
6015
6017
6019
6021
6023
6025
6027


28037
28039
28041
28043
28045
28047
28049
28051
28053
28055
28057
28059
28061
28063
28065
28067
28069
28071
28073
28075
28077
28079
28081
28083
28085
28087
28089
28091
28093
28095
28097
28099
28101
28103
28105
28107
28109
28111
28113
28115
28117
28119
28121
28123
28125
28127
28129
28131
28133
28135
28137
28139
28141
28143
28145
28147
28149
28151
28153
28155
28157
28159
28161
28163
29001
29003
29005
29007
29009
29011
29013
29015
29017
29019
29021
29023
29025
29027
29029
29031
29033
29035
29037
29039
29041
29043
29045
29047
29049
29051
29053
29055
29057
29059
29061
29063
29065
29067
29069
29071
29073
29075
29077
29079
29081
29083
29085
29087
29089
29091
29093
29095
29097
29099
29101
29103
29105
29107
29109
29111
29113
29115
29117
29119
29121
29123
29125
29127
29129
29131
29133
29135
29137
29139
29141
29143
29145
29147
29149
29151
29153
29155
29157
29159
29161
29163
29165
29167
29169
29171
29173
29175
29177
29179
29181
29183
29185
29186
29187
29189
29195
29197
29199
29201
29203
29205
2920

49017
49019
49021
49023
49025
49027
49029
49031
49033
49035
49037
49039
49041
49043
49045
49047
49049
49051
49053
49055
49057
50001
50003
50005
50007
50009
50011
50013
50015
50017
50019
50021
50023
50025
50027
51001
51003
51005
51007
51009
51011
51013
51015
51017
51019
51021
51023
51025
51027
51029
51031
51033
51035
51036
51037
51041
51043
51045
51047
51049
51051
51053
51057
51059
51061
51063
51065
51067
51069
51071
51073
51075
51077
51079
51081
51083
51085
51087
51089
51091
51093
51095
51097
51099
51101
51103
51105
51107
51109
51111
51113
51115
51117
51119
51121
51125
51127
51131
51133
51135
51137
51139
51141
51143
51145
51147
51149
51153
51155
51157
51159
51161
51163
51165
51167
51169
51171
51173
51175
51177
51179
51181
51183
51185
51187
51191
51193
51195
51197
51199
51510
51515
51520
51530
51540
51550
51570
51580
51590
51595
51600
51610
51620
51630
51640
51650
51660
51670
51678
51680
51683
51685
51690
51700
51710
51720
51730
51735
51740
51750
51760
51770
51775
51790
51800
51810
5182

In [58]:
County_NO2.to_csv('County_NO2_Filled.csv')

In [59]:
#Now doing CO
############################ NOTE: We only include PPM Points here, to make sure data is uniform, 
#Total entries: 253181
#PPM entries: 243346
#PPB entries: 9835
CO = Air_Qual.loc[Air_Qual['CO_Unit'].isin(['PPM'])]
CO = CO.drop(columns=['PM10_AQI','PM25_AQI','OZONE_AQI', 'NO2_AQI','SO2','SO2_Unit'])
CO = CO.astype({'CO': 'float64'})

#Drop unit column, everythin is PPM
CO = CO.drop(columns=['CO_Unit'])
CO = CO.groupby(['Latitude','Longitude','ValidDate']).mean().reset_index()
County_CO = BaseFrame.copy()
County_CO['CO'] = County_PM25.apply(lambda x: air_qual_county(x['FIPS'], x['Date'], CO, 'CO', County_Centers, neighbors=10), axis = 1)


1001
1003
1005
1007
1009
1011
1013
1015
1017
1019
1021
1023
1025
1027
1029
1031
1033
1035
1037
1039
1041
1043
1045
1047
1049
1051
1053
1055
1057
1059
1061
1063
1065
1067
1069
1071
1073
1075
1077
1079
1081
1083
1085
1087
1089
1091
1093
1095
1097
1099
1101
1103
1105
1107
1109
1111
1113
1115
1117
1119
1121
1123
1125
1127
1129
1131
1133
2013
2016
2020
2050
2060
2068
2070
2090
2100
2105
2110
2122
2130
2150
2164
2170
2180
2185
2188
2195
2198
2220
2230
2240
2261
2270
2275
2282
2290
4001
4003
4005
4007
4009
4011
4012
4013
4015
4017
4019
4021
4023
4025
4027
5001
5003
5005
5007
5009
5011
5013
5015
5017
5019
5021
5023
5025
5027
5029
5031
5033
5035
5037
5039
5041
5043
5045
5047
5049
5051
5053
5055
5057
5059
5061
5063
5065
5067
5069
5071
5073
5075
5077
5079
5081
5083
5085
5087
5089
5091
5093
5095
5097
5099
5101
5103
5105
5107
5109
5111
5113
5115
5117
5119
5121
5123
5125
5127
5129
5131
5133
5135
5137
5139
5141
5143
5145
5147
5149
6001
6003
6005
6007
6009
6011
6013
6015
6017
6019
6021
6023
6025
6027


28037
28039
28041
28043
28045
28047
28049
28051
28053
28055
28057
28059
28061
28063
28065
28067
28069
28071
28073
28075
28077
28079
28081
28083
28085
28087
28089
28091
28093
28095
28097
28099
28101
28103
28105
28107
28109
28111
28113
28115
28117
28119
28121
28123
28125
28127
28129
28131
28133
28135
28137
28139
28141
28143
28145
28147
28149
28151
28153
28155
28157
28159
28161
28163
29001
29003
29005
29007
29009
29011
29013
29015
29017
29019
29021
29023
29025
29027
29029
29031
29033
29035
29037
29039
29041
29043
29045
29047
29049
29051
29053
29055
29057
29059
29061
29063
29065
29067
29069
29071
29073
29075
29077
29079
29081
29083
29085
29087
29089
29091
29093
29095
29097
29099
29101
29103
29105
29107
29109
29111
29113
29115
29117
29119
29121
29123
29125
29127
29129
29131
29133
29135
29137
29139
29141
29143
29145
29147
29149
29151
29153
29155
29157
29159
29161
29163
29165
29167
29169
29171
29173
29175
29177
29179
29181
29183
29185
29186
29187
29189
29195
29197
29199
29201
29203
29205
2920

49017
49019
49021
49023
49025
49027
49029
49031
49033
49035
49037
49039
49041
49043
49045
49047
49049
49051
49053
49055
49057
50001
50003
50005
50007
50009
50011
50013
50015
50017
50019
50021
50023
50025
50027
51001
51003
51005
51007
51009
51011
51013
51015
51017
51019
51021
51023
51025
51027
51029
51031
51033
51035
51036
51037
51041
51043
51045
51047
51049
51051
51053
51057
51059
51061
51063
51065
51067
51069
51071
51073
51075
51077
51079
51081
51083
51085
51087
51089
51091
51093
51095
51097
51099
51101
51103
51105
51107
51109
51111
51113
51115
51117
51119
51121
51125
51127
51131
51133
51135
51137
51139
51141
51143
51145
51147
51149
51153
51155
51157
51159
51161
51163
51165
51167
51169
51171
51173
51175
51177
51179
51181
51183
51185
51187
51191
51193
51195
51197
51199
51510
51515
51520
51530
51540
51550
51570
51580
51590
51595
51600
51610
51620
51630
51640
51650
51660
51670
51678
51680
51683
51685
51690
51700
51710
51720
51730
51735
51740
51750
51760
51770
51775
51790
51800
51810
5182

In [60]:
County_CO.to_csv('County_CO_Filled.csv')

In [61]:
#Now doing SO2
############################ NOTE: We only include PPB Points here, to make sure data is uniform, 
#Total entries: 321946
#PPM entries: 6714
#PPB entries: 315232

SO2 = Air_Qual.loc[Air_Qual['SO2_Unit'].isin(['PPM','PPB'])]
SO2 = SO2.drop(columns=['PM10_AQI','PM25_AQI','OZONE_AQI', 'NO2_AQI','CO','CO_Unit'])
SO2 = SO2.astype({'SO2': 'float64'})

#Drop unit column, everythin is PPB
SO2 = SO2.drop(columns=['SO2_Unit'])
SO2 = SO2.groupby(['Latitude','Longitude','ValidDate']).mean().reset_index()
County_SO2 = BaseFrame.copy()
County_SO2['SO2'] = County_PM25.apply(lambda x: air_qual_county(x['FIPS'], x['Date'], SO2, 'SO2', County_Centers, neighbors=10), axis = 1)


1001
1003
1005
1007
1009
1011
1013
1015
1017
1019
1021
1023
1025
1027
1029
1031
1033
1035
1037
1039
1041
1043
1045
1047
1049
1051
1053
1055
1057
1059
1061
1063
1065
1067
1069
1071
1073
1075
1077
1079
1081
1083
1085
1087
1089
1091
1093
1095
1097
1099
1101
1103
1105
1107
1109
1111
1113
1115
1117
1119
1121
1123
1125
1127
1129
1131
1133
2013
2016
2020
2050
2060
2068
2070
2090
2100
2105
2110
2122
2130
2150
2164
2170
2180
2185
2188
2195
2198
2220
2230
2240
2261
2270
2275
2282
2290
4001
4003
4005
4007
4009
4011
4012
4013
4015
4017
4019
4021
4023
4025
4027
5001
5003
5005
5007
5009
5011
5013
5015
5017
5019
5021
5023
5025
5027
5029
5031
5033
5035
5037
5039
5041
5043
5045
5047
5049
5051
5053
5055
5057
5059
5061
5063
5065
5067
5069
5071
5073
5075
5077
5079
5081
5083
5085
5087
5089
5091
5093
5095
5097
5099
5101
5103
5105
5107
5109
5111
5113
5115
5117
5119
5121
5123
5125
5127
5129
5131
5133
5135
5137
5139
5141
5143
5145
5147
5149
6001
6003
6005
6007
6009
6011
6013
6015
6017
6019
6021
6023
6025
6027


28037
28039
28041
28043
28045
28047
28049
28051
28053
28055
28057
28059
28061
28063
28065
28067
28069
28071
28073
28075
28077
28079
28081
28083
28085
28087
28089
28091
28093
28095
28097
28099
28101
28103
28105
28107
28109
28111
28113
28115
28117
28119
28121
28123
28125
28127
28129
28131
28133
28135
28137
28139
28141
28143
28145
28147
28149
28151
28153
28155
28157
28159
28161
28163
29001
29003
29005
29007
29009
29011
29013
29015
29017
29019
29021
29023
29025
29027
29029
29031
29033
29035
29037
29039
29041
29043
29045
29047
29049
29051
29053
29055
29057
29059
29061
29063
29065
29067
29069
29071
29073
29075
29077
29079
29081
29083
29085
29087
29089
29091
29093
29095
29097
29099
29101
29103
29105
29107
29109
29111
29113
29115
29117
29119
29121
29123
29125
29127
29129
29131
29133
29135
29137
29139
29141
29143
29145
29147
29149
29151
29153
29155
29157
29159
29161
29163
29165
29167
29169
29171
29173
29175
29177
29179
29181
29183
29185
29186
29187
29189
29195
29197
29199
29201
29203
29205
2920

49017
49019
49021
49023
49025
49027
49029
49031
49033
49035
49037
49039
49041
49043
49045
49047
49049
49051
49053
49055
49057
50001
50003
50005
50007
50009
50011
50013
50015
50017
50019
50021
50023
50025
50027
51001
51003
51005
51007
51009
51011
51013
51015
51017
51019
51021
51023
51025
51027
51029
51031
51033
51035
51036
51037
51041
51043
51045
51047
51049
51051
51053
51057
51059
51061
51063
51065
51067
51069
51071
51073
51075
51077
51079
51081
51083
51085
51087
51089
51091
51093
51095
51097
51099
51101
51103
51105
51107
51109
51111
51113
51115
51117
51119
51121
51125
51127
51131
51133
51135
51137
51139
51141
51143
51145
51147
51149
51153
51155
51157
51159
51161
51163
51165
51167
51169
51171
51173
51175
51177
51179
51181
51183
51185
51187
51191
51193
51195
51197
51199
51510
51515
51520
51530
51540
51550
51570
51580
51590
51595
51600
51610
51620
51630
51640
51650
51660
51670
51678
51680
51683
51685
51690
51700
51710
51720
51730
51735
51740
51750
51760
51770
51775
51790
51800
51810
5182

In [62]:
County_SO2.to_csv('County_SO2_Filled.csv')

In [63]:
Air_Qual_Filled = County_Ozone.merge(County_PM10, on = ['FIPS','Date']).merge(County_PM25, on = ['FIPS','Date']).merge(County_NO2, on = ['FIPS','Date']).merge(County_CO, on = ['FIPS','Date']).merge(County_SO2, on = ['FIPS','Date'])

In [64]:
Air_Qual_Filled.to_csv('Air_Qual_Filled.csv')
Air_Qual_Filled.head(100)

,FIPS,Date,OZONE_AQI,PM10_AQI,PM25_AQI,NO2_AQI,CO,SO2
0,1001,01/01/20,16.422424,10.125737,54.500000,7.468020,0.287662,0.428571
1,1001,01/02/20,22.169518,10.668188,54.812500,13.391453,0.326020,0.583333
2,1001,01/03/20,17.153856,10.328083,29.875000,12.347675,0.353808,0.133333
3,1001,01/04/20,25.211669,10.220940,20.312500,4.207851,0.214592,0.000000
4,1001,01/05/20,18.645173,9.011376,35.875000,6.152528,0.252093,0.066667
5,1001,01/06/20,18.775524,13.742528,55.625000,13.197471,0.306029,0.285714
6,1001,01/07/20,24.574919,17.555970,41.187500,10.990515,0.273448,0.000000
7,1001,01/08/20,21.638274,19.046460,41.437500,14.172449,0.316060,7.153846
8,1001,01/09/20,35.832824,22.365232,63.812500,14.402165,0.402442,1.733333
9,1001,01/10/20,33.578640,17.690726,54.562500,11.318338,0.312424,0.000000
